In [1]:
import keras
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from itertools import count
from sklearn.metrics import accuracy_score
from keras.datasets import cifar10
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import EarlyStopping, CSVLogger
from scipy.stats import pearsonr
from tqdm import tqdm
from IPython.display import clear_output

Using TensorFlow backend.


In [0]:
BATCH_SIZE = 128
EPOCHS = 9999
IMAGE_SIZE = 32
NUM_CLASSES = 10
NUM_CHANNELS = 3
MODEL_NAME = "CIFAR_baseline"
PATH = ""
NR_OF_RUNS = 10

# Set seeds

# Preprocess

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#x_train = preprocess(x_train)
#x_test = preprocess(x_test)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 11s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_trainc = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_testc = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [0]:
x_train_full = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train_full /= 255
x_test /= 255

# Model

In [0]:
def CIFARmodel(imsize, num_classes, num_channels):
    inputs = Input((imsize,imsize,num_channels))
    
    x = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(inputs)
    x = Conv2D(filters=64, kernel_size=(3,3), strides=2)(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(x)
    
    x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', strides=2, padding='same')(x)
    x = Conv2D(filters=128, kernel_size=(3,3), activation='relu', strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(filters=128, kernel_size=(1,1), activation='relu', padding='valid')(x)
    x = Conv2D(filters=10, kernel_size=(1,1),strides=(1,1), padding='valid')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    optimizer = keras.optimizers.Adam(learning_rate = 1e-04)

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    return model



# Train

In [7]:
for run in range(1, NR_OF_RUNS+1):
    # Split the data
    x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_trainc, test_size=0.20, shuffle= True)
    
    models = []
    accuracies = []
    predictions = []
    print(f"\n ===== Train model: Baseline: Run: {run}  =====")
        
    # Set the seeds
    np.random.seed(run*31)
    tf.random.set_seed(run*31)

    # Create directories
    os.makedirs(PATH + MODEL_NAME + f"/{run}/history", exist_ok=True)
    os.makedirs(PATH + MODEL_NAME + f"/{run}/weights", exist_ok=True)
        
    # weight init method
    model = CIFARmodel(IMAGE_SIZE,NUM_CLASSES,NUM_CHANNELS)
            
    #save weights 
    weights_path = PATH + MODEL_NAME + f"/{run}/weights/weights-baseline.h5"
            
    if os.path.exists(weights_path):
        print(f"Skipping training of model: weights exists")
        model.load_weights(weights_path)
    else:
        # initiate early stopping
        es = EarlyStopping(min_delta=0.01, patience=3)
        csv_logger = CSVLogger(PATH + MODEL_NAME + f"/{run}/history/history.csv", separator=';')
        # train
        model.fit(x_train,y_train,
                batch_size = BATCH_SIZE,
                epochs = EPOCHS,
                validation_data = (x_val,y_val),
                shuffle = True,
                callbacks=[es, csv_logger])
        model.save_weights(weights_path)
            
    acc = model.evaluate(x_test,y_testc)[1]
    print(f"Run: {run} added. Resulting score: {acc}")

    print("\n ===== Saving results =====")  
    # Save the results
    file = PATH + MODEL_NAME + f"/results_.csv"
    df = pd.DataFrame([[run,acc]])
    if not os.path.isfile(file):
        df.to_csv(file, header=["run","accuracy"], index=False)
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False)
    clear_output(wait=True)




 ===== Train model: Baseline: Run: 10  =====
Train on 40000 samples, validate on 10000 samples
Epoch 1/9999
40000/40000 [==============================] - 10s 242us/step - loss: 1.7372 - accuracy: 0.3860 - val_loss: 2.3212 - val_accuracy: 0.1073
Epoch 2/9999
40000/40000 [==============================] - 9s 228us/step - loss: 1.3311 - accuracy: 0.5338 - val_loss: 1.5128 - val_accuracy: 0.4465
Epoch 3/9999
40000/40000 [==============================] - 9s 228us/step - loss: 1.1638 - accuracy: 0.5937 - val_loss: 1.2326 - val_accuracy: 0.5612
Epoch 4/9999
40000/40000 [==============================] - 9s 228us/step - loss: 1.0454 - accuracy: 0.6373 - val_loss: 1.1171 - val_accuracy: 0.5995
Epoch 5/9999
40000/40000 [==============================] - 9s 229us/step - loss: 0.9573 - accuracy: 0.6720 - val_loss: 1.1868 - val_accuracy: 0.5807
Epoch 6/9999
40000/40000 [==============================] - 9s 229us/step - loss: 0.8789 - accuracy: 0.6974 - val_loss: 1.0727 - val_accuracy: 0.6263
Epo

# Results

## Accuracy
The final accuracy of the ensamble on the test set

In [8]:
!zip -r /content/CIFAR_baseline.zip /content/CIFAR_baseline

  adding: content/CIFAR_baseline/ (stored 0%)
  adding: content/CIFAR_baseline/9/ (stored 0%)
  adding: content/CIFAR_baseline/9/weights/ (stored 0%)
  adding: content/CIFAR_baseline/9/weights/weights-baseline.h5 (deflated 10%)
  adding: content/CIFAR_baseline/9/history/ (stored 0%)
  adding: content/CIFAR_baseline/9/history/history.csv (deflated 47%)
  adding: content/CIFAR_baseline/7/ (stored 0%)
  adding: content/CIFAR_baseline/7/weights/ (stored 0%)
  adding: content/CIFAR_baseline/7/weights/weights-baseline.h5 (deflated 10%)
  adding: content/CIFAR_baseline/7/history/ (stored 0%)
  adding: content/CIFAR_baseline/7/history/history.csv (deflated 48%)
  adding: content/CIFAR_baseline/8/ (stored 0%)
  adding: content/CIFAR_baseline/8/weights/ (stored 0%)
  adding: content/CIFAR_baseline/8/weights/weights-baseline.h5 (deflated 10%)
  adding: content/CIFAR_baseline/8/history/ (stored 0%)
  adding: content/CIFAR_baseline/8/history/history.csv (deflated 49%)
  adding: content/CIFAR_baseli